In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
import re

In [3]:
input_file = "angst-data/input3.txt"
file = open(input_file, "r") 
text = file.read() 
print('corpus length:', len(text))

corpus length: 1053989


In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
chars

total chars: 88


['\t',
 '\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~']

In [5]:
## Chopping the text up into strings of 40 characters to train on
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 351317


In [6]:
## Vectorizing our sequences
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [7]:
## We're designing our model here- choosing the number of layers and such
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Showing the poetry quality at the end of each epoch
def on_epoch_end(epoch, logs):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        ## What we're doing is giving the bot 40 characters and asking
        ## it to predict the 41st character, over and over
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [9]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

## Here's where we train the model on our data
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
351232/351317 [============================>.] - ETA: 0s - loss: 2.0585
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "th the interstate smooth as needles glea"
th the interstate smooth as needles gleath and the searth the shade the black the could the sand and the could the sun the sand and the to start of the sears, 

I was the would the soul the sight of the searth the can the soul the close and the seath the could the black the side the hand the disent and the seath the thint of the sand the searth the start of the searth the soul the could the can a wast and the sand the soul the searth, 

----- diversity: 0.5
----- Generating with seed: "th the interstate smooth as needles glea"
th the interstate smooth as needles gleant every such that the place and the sea world of the flase of the bought to flame the aire
and dis and the donat and cours the start the flace of the doon and could a light of the saw she past how the are of the li

/home/Randus/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 ]bl~]___.

The sayed the stand and the stand and the st_z_.


I stil_ of~, the say~bid~bes the seems and the so~bes and t]b_.

~bes and the se_ze the st_zed _blow of th~m.

]ver and s~bbite and~.

I was h~bide the _bes 	ble and the_ t_.

The sou~b_bleanting the sou	-step and the she and _bes+;
_ked and the _blac	, and the s~bbess of t]zed _bite
and the st_zed and the spe_
----- diversity: 0.5
----- Generating with seed: " he must conquer the ancient

secret
tha"
 he must conquer the ancient

secret
that_: _zzing~_~ves of a]_ in], _____. I see the__b]moth~ble _
and the longer the sun because home. 

So_- d]'t ~_zed the +w_k~~_
I the land  


And the s~bless and the moth_p in]bemate
_ker, bod__ 
the_ wi~ker less o~k~bubut, ~wable streets are i~
_g_zer still down his_pho	b~cally of the co	, s~bleping _zed,
	bidis_, a~sp~, 
The hand ~wa~b+bal~s.

I say he says of the su_.
I am better let th~m, 
Th~
----- diversity: 1.0
----- Generating with seed: " he must conquer the ancient

secret
tha"

t ae tkniss  dest f a mie o t faie oe steae  e  epen a don t w and bar ams could at. It. aead uen foret aent as amp aared things ae wa e iee apl aamo to a p aa less aete de and  add a r aar paiaral an aspee tona
l  
A alvaed I aar a aa d aak se bove ape
----- diversity: 1.2
----- Generating with seed: "knowledge might enrich my mind?
Since th"
knowledge might enrich my mind?
Since th a a a a e  a raa a ara am ia ares es.

cane atens aapa ac l a t age ce wichiet  aare e ad
 ae c ow y too ahd mae aat pin aaras
 
bl in hi prot sl fea art ere l t aa tousand whaara ver la e weelia mer re eye er abe 
e  ag aa l daye ex an  a and ara as ttromin al aate aeve sies, an are e b nigh t day in andre and ares
 a a lik the aiveee ane s a  th   
eems d  eye elloas e aare age,
eser are a end 
351317/351317 [==============================] - 534s 2ms/step - loss: 4.8704
Epoch 8/60
351232/351317 [============================>.] - ETA: 0s - loss: 4.7386
----- Generating text after Epoch: 7
----- diversity

a fine method, if you lived in a counte  at ata aae  t te  ane e aa  nene  t ate a aaa aa  aana aa t a  aae atene ate aten aa at a  t aa aat t a ate ae a at aa  aae at ate aa a  t aaatee a ate aat te at e  t a ae aat ate aa atate at an a aa ate ate at ane aa te a a a aaa aa ate a a aaana aaa a aa at ae  ate a a aaa  aa aa  an aa aa aaa a a e aate a ta t aaa aaa ate e  a aa  aaa a aaa a ane aaa t aa aaate a ata a ae  aa a aa aa aa aten
----- diversity: 0.5
----- Generating with seed: ",
a fine method, if you lived in a count"
,
a fine method, if you lived in a counte an at aa at e atee at aa aa ann at t ata a ate a t aa a a aa aa aa aan a aaa aa aaa  aa a atee ae ae e  ue aee aa a  ae ae e at aa t te a a aee  a aatee ata a ate e at aa aaee aa ate aa a  t aa a atee a a aa ate aa ae aat aae aaat a aaa ate aat aa a taa  t a  t aa  an a a ae aa aaa a aaa aaa aa a aat aa a  aa te t ae  aa  anen a at a a aae  aa a ate aat aa atenee a aee a a at t a e ate at at aaa
----- diversity: 1.0
----- G

...

or as crystals spilled from t t e  ne   o t    t tin o te   o  ano  t n   o  tn  n s ten  t ton   t nn  o  noe te  t    aon     t      o  t  ane  n t  or  o  a   oze t an   to t o t    r t tne on t t e     t rn an  n t n n t ne t n t t o t   tr t t  tr t ao a ne t se n   te   o t t t   n  a t  tet  t     a  t st n  t  te in  onn t  on t  s on t  t  ne  e enn e     t  t t t t e tn t n t   o  t t e  e  t an e    t ne   tt   e  
----- diversity: 0.5
----- Generating with seed: "eidoscope,
or as crystals spilled from t"
eidoscope,
or as crystals spilled from t t an i o   tn ete t tt  in   an r    rernnee r r t n t   io nttan t t tenar rn t ran o  oe snaroe o  s a i nn  n  o    ots n  a  e a t o rer t te e o ot  nn e sr r   ta  io t re e e ta  t     ine i terns r it t   titsare ine e nr  t   a e rtr net t tt r aor r tnno o a t r  io   ree  t eoeot  ro t o ostt  t ne t  s onon  s o t t  r  or  ar  t tst ro strre o r n inne t  e t  sn  t at tasn rinsnee s
----- diversity: 1.0
----- Generating

but what you sang waae e nnrenei inost e sn a n e ar  en ie eie n serene iit it teet teasa  anniee a inannee ti r e snin anenosits ee ea   o tt a asenoi  re  eantansesee oeoi i nrr o eaie teee ase  esne s ot ae i risseir atraenonee asa oretrraiieneri   tst oatnnonoiee to t  e  t noinoetsareee r  s ieot antt eaan i noe tetire ettnern sn  sinettean sesae o e oneot eeeiieno n to rtti io s ies   oi e tn an ii ri enttnoee
----- diversity: 1.2
----- Generating with seed: " said was one thing
but what you sang wa"
 said was one thing
but what you sang wa  en i oort  o nninoon te an nseiiasna  sn oit ooeae t ai nteoo ssi n oe nt rasniineitiiiatton  trananoisetrr teett   rr rr r etote ens a a ensitssino  oierte ie nes ee osertossriarerot iertraarrsnarias osra reasr ts neeoee oi eo i norere oare tonat  n aet erennerrt on  nroseine  neronea trteto o tntaeerisneiiinnoaisons i ene tsietns ntt nn easitr s ssa sher ars senitesarn oen  ensianont arr n e a
351317/351317 [==============================]

their seeds. Tick. Black    e te t  t  se  at  ne  a t      n       e t    n  se ee te te te a t ed   te a   to   o r aoe  a t   ne n   a       eo s  ata   o e   t w  n t e          r o ta   t  a   s   te   te   t e   a  an   a    tar     ae    o  a  oo n o t t n    n  t oo ato n   oee       a  t ne     te      t   e to    t ne e  e a  a     a t e e  te      a o a   e   te e   g   tt       a  t    a s  a e ne    a  oae  
----- diversity: 0.5
----- Generating with seed: "loved spitting

their seeds. Tick. Black"
loved spitting

their seeds. Tick. Black  tse t   to ne  a g s     s n  ioteraa e a  e ontoe s ooeannin tn a t s  aooa  at -  t t s  aaat e ee  onoe i e  se  ne tts ar ae ar tote ton an e a tre ten    e et n t  aet s titrtors ttt n eo a   ae   not aai  nese    oe a a oea ae e rxo  s or iei sne x se  sne t t tase ae n e e ana  n ne t     rsr as e ars n  taset te  ne    eo  n te aonee s te   s oet  a a t ton nsie e  it   o  norenre s t n 
----- diversity: 1.0
----- Generating with

you could reono t rirt sa osr  e   r s tn iens  ta st nana   arn t nnesees  sa t tsaaett ao tirooeo ante a etir t oesstoo  eo esenttts ni  oens eennet  ntnes  einios r tir ots i ee  t tttero rne  r  tre a t    iosesaons  aa ne i sne ino osort s oreione nr  r io  itt nrtnsaete oeostrs e soee ora   e sns oe n tsi eareeni s eaieste s rore se r t toi oaneeareree   a rreonn at e s rtssr  eentatasreo r n i eitres
----- diversity: 1.2
----- Generating with seed: "nk of all the different ways

you could "
nk of all the different ways

you could r   esr   i ee tinn i  teae or ssae tt tms t reo ao aoieoeas ieeio  ire ea n   atee e ti aeoenrea  a  nst ens str i no eeeseeoontiie e storen too   osao etooees  ns t o hieio  otre se oer   ri e a oa  o o s sssiniso t anitnrer nann ast res  srsti er sain idee s aeoose oias se eato iaen tt   ato enieioeaasoai stesoeiiie  ra  asneirseer  as ate nii    ee  e seo ainirtn   rot asotsna o t  n ott  tini
351317/351317 [==============================] - 495s 1m

of three-way speakers,   so  e       teee  ee  t eo se n s t a t a tee    o a o t ee s  a   t t t e t ab t t   e  t  t  n   t oe  soae t t  es  te  e a e  e t te  s as s   te t  oee  t  t t te tt a o   t a an   oe e e  n ot  o t e s   ne oonta n   te  e t    t   e st  t   oo  teg a     oet  ee  t  t  oe t  e  t o fin t   i  en  t toea  t tm ae ot  t te saet   as  o t a  t  oo  t t a tee  tt   ta   e t t  o n t  ane   st
----- diversity: 0.5
----- Generating with seed: "neer amp, a pair
of three-way speakers, "
neer amp, a pair
of three-way speakers, a  aernte asoe rs noi s er ooe as r o  ai   ois tasa   oett t   otese eo atee n to ost   t is  tt on ta  teet t e n orn aset a ran o saene  o  s r  iste rns  t i tsnatse  t se otte o t  os  o tt etes nnt  ant  e ae ana eo aee rt ioeee n e sn e tono io nnee tinerineata   itn   one   to tn t onraass ini a  t tiaete oren   o tneiet n t toa oaas    tto e t s ne   a se  e   rete t  n no o    io  se  t 
----- diversity: 1.0
----- Generating with 

In [10]:
model.save('angst3.h5')

In [16]:
## Here is where you can actually use the model on a given piece of text!

for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = 'Along the shore the cloud waves break, t'
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

----- diversity: 0.2
----- Generating with seed: "Along the shore the cloud waves break, t"
Along the shore the cloud waves break, the sea-equas

The swands the strange my story the sun and wall

The still of strangers and strangers and star;
Is the strange the sun and some bright and silent man born

and silent the shades of my hearth is shades of the strange

and so the strange the strange the strange the sigh
the heart and a strange the side, 

The strange my seasones and start, the strange the dream 
The sigh the side of the same sighs the strain 

The silvers of the strange the sighs the strange the strain 

The silking a strange the strain the strange a strange the strange
the strange and shadow spired the strange

but a part of the strange the strange a sound,
and some watch the sun in the strange a strange the sea,

The silking to the strange the sun, 

The sighs the sidewarded of the season and silent black stare.

The silver seems the seat the mother short

whist the sun in t

/home/marykate/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


he seemed strain and summer slight good of the stopped the sunsetings
What have have to see the strange in sun love
it grasp best a life start bepper that was the window

the world of the patiaus and silent beated to see a good. 

The they connection, some fight and small water,

where light as if seem. 

What he sav, 

----- diversity: 1.0
----- Generating with seed: "Along the shore the cloud waves break, t"
Along the shore the cloud waves break, the tappore,
One op along as hid bringed got on, 
And only of spience is, phosogless in in thework mome must,
an chance we nights in she was take into a age. 
as their in. I want on pitterers

Or lear out with fly, As the sun, where a across a clastales
of turn. New of strange
    And night sew he would work
and camifrmbity young down the strumme the big to teat ot,
I
can be my heacy in a scide
That smother of you widnes in your hught
A diffor his elishoush of light
farther, people asleeps somef,
Incaus, sourds and sunk ways; 
Oblate farmits